In [ ]:
%%capture
!pip install laspy
!pip install rasterio
!pip install alphashape shapely
!pip install open3d

In [ ]:
import glob

import tqdm
import geopandas as gpd
import laspy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
import scipy.stats
import seaborn as sns
import shapely
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, RocCurveDisplay
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import laspy
import scipy.stats

In [ ]:
file_paths = glob.glob("/kaggle/input/hutechaichallenge2024-mc/Train/*/*.las")
file_paths.sort()

In [ ]:
data = pd.DataFrame({"path": file_paths})
data["species"] = data["path"].map(lambda p: p.split("/")[-2])
data.sample(n=3)

# Compute features

In [ ]:
# Step 1: Find the highest tree (Ptop)
def find_highest_tree(points):
    idx = np.argmax(points[:, 2])  # Find point with max Z
    return points[idx]

# Step 2: Rotate profile and extract edge points
def extract_tree_edges(points, Ptop, num_segments=36):
    angles = np.linspace(0, np.pi, num_segments)  # Rotate from 0° to 180°
    edge_points = []
    
    for angle in angles:
        # Rotate points around Ptop
        rot_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        rotated_xy = (points[:, :2] - Ptop[:2]) @ rot_matrix.T + Ptop[:2]
        rotated_points = np.column_stack((rotated_xy, points[:, 2]))
        
        # Find the convex hull
        hull = ConvexHull(rotated_points[:, :2])
        edge_points.append(rotated_points[hull.vertices])
    
    return edge_points

# Step 3: Cluster trees based on extracted edges
def segment_trees(points, edge_points, threshold=2.0):
    clustered = np.zeros(len(points), dtype=int)
    cluster_id = 1
    
    for edges in edge_points:
        for edge in edges:
            distances = np.linalg.norm(points[:, :2] - edge[:2], axis=1)
            clustered[distances < threshold] = cluster_id
        cluster_id += 1
    
    return clustered.tolist()

In [ ]:
def extract_tree_crown_keypoints(points, tree_top, dis):
    """
    Extract keypoints of the tree crown based on the steps provided.

    Args:
        points (numpy.ndarray): Array of points (N, 3) representing the point cloud.
        tree_top (tuple): (x, y, z) coordinates of the tree top vertex.
        dis (float): Average point spacing distance.

    Returns:
        numpy.ndarray: Keypoints of the tree crown.
    """
    # 1. Draw the vertical line L from the tree top to the ground
    x_top, y_top, z_top = tree_top
    z_min = np.min(points[:, 2])  # Ground level (minimum Z value)
    
    # 2. Calculate the width D
    D = 2 * dis  # Width of the profile around the tree top
    
    # 3. Draw parallel lines with spacing d
    d = dis  # Can use the same value for spacing
    keypoints = []

    # Iterate through several levels from tree top to ground
    num_lines = int((z_top - z_min) / d)  # Number of lines along the Z-axis
    for i in range(num_lines):
        z_level = z_top - i * d  # Current level of the parallel line
        # Find points around the current Z level within the profile width D
        points_at_level = points[(points[:, 2] >= z_level - D / 2) & 
                                  (points[:, 2] <= z_level + D / 2) & 
                                  (np.abs(points[:, 0] - x_top) <= D / 2) & 
                                  (np.abs(points[:, 1] - y_top) <= D / 2)]
        
        # Collect keypoints
        keypoints.append(points_at_level)
    
    keypoints = np.vstack(keypoints) if len(keypoints) > 0 else np.array([])
    return keypoints

def plot_tree_crown(points, keypoints, tree_top):
    """
    Plot the points and keypoints of the tree crown.

    Args:
        points (numpy.ndarray): Original point cloud.
        keypoints (numpy.ndarray): Keypoints of the tree crown.
        tree_top (tuple): (x, y, z) coordinates of the tree top vertex.
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot all points
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=1, label="Point Cloud")
    
    # Plot keypoints
    ax.scatter(keypoints[:, 0], keypoints[:, 1], keypoints[:, 2], color='red', s=10, label="Tree Crown Keypoints")
    
    # Mark the tree top
    ax.scatter(tree_top[0], tree_top[1], tree_top[2], color='green', s=50, label="Tree Top")
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.title("Tree Crown Keypoints Extraction")
    plt.show()

In [ ]:
import alphashape
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
import open3d as o3d

def compute_alpha_shape(point_cloud, alpha=0.1):
    """
    Dựng biên của đám mây điểm bằng thuật toán Alpha Shapes.

    Args:
        point_cloud (numpy.ndarray): Đám mây điểm (N, 3) chứa (x, y, z).
        alpha (float): Giá trị alpha để điều chỉnh độ chặt chẽ của biên.

    Returns:
        Polygon hoặc MultiPolygon: Đường biên của tán cây.
    """
    # Chỉ lấy tọa độ X, Y (bỏ Z vì biên là 2D)
    xy_points = point_cloud[:, :2]

    # Tạo Alpha Shape
    alpha_shape = alphashape.alphashape(xy_points, alpha)

    return alpha_shape

In [ ]:
import matplotlib.pyplot as plt

def plot_alpha_shape(point_cloud, alpha_shape):
    """
    Vẽ đường biên của tán cây dựa trên Alpha Shapes.

    Args:
        point_cloud (numpy.ndarray): Đám mây điểm (N, 3).
        alpha_shape (Polygon hoặc MultiPolygon): Đường biên.
    """
    plt.figure(figsize=(8, 8))
    plt.scatter(point_cloud[:, 0], point_cloud[:, 1], s=1, label="Điểm")
    
    if isinstance(alpha_shape, Polygon):
        x, y = alpha_shape.exterior.xy
        plt.plot(x, y, 'r-', linewidth=2, label="Alpha Shape")
    elif isinstance(alpha_shape, MultiPolygon):
        for poly in alpha_shape.geoms:
            x, y = poly.exterior.xy
            plt.plot(x, y, 'r-', linewidth=2, label="Alpha Shape")

    plt.legend()
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Alpha Shape của Tán Cây")
    plt.show()

In [ ]:
import laspy
import numpy as np
from scipy.spatial import ConvexHull

def compute_cloud_features(path: str, *, height_threshold: float = 1.0) -> dict:
    las = laspy.read(path)
    out = {}
    
    points = np.stack((las.x, las.y, las.z), axis=1)
    
    height_mask = points[:, 2] > height_threshold
    points = points[height_mask]

    # ĐẶc trưng tọa độ
    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()
    z_min, z_max = points[:, 2].min(), points[:, 2].max()
    mean_x, mean_y, mean_z = points[:, 0].mean(), points[:, 1].mean(), points[:, 2].mean()
    var_x, var_y, var_z = points[:, 0].var(), points[:, 1].var(), points[:, 2].var()

    # Bounding Box (Hình chữ nhật)
    width = x_max - x_min
    depth = y_max - y_min
    height = z_max - z_min

    # 📌 Shape Ratio
    vertical_proportion = float(height) / (width + depth)

    # 📌 Mật độ điểm
    density_xy = len(points) / ((x_max - x_min) * (y_max - y_min))
    density_yz = len(points) / ((y_max - y_min) * (z_max - z_min))
    density_xyz = len(points) / ((x_max - x_min) * (y_max - y_min) * (z_max - z_min))
    point_density_ratio = density_xy / density_xyz  # 📌 Tỷ lệ mật độ điểm 2D/3D

    # 📌 Đặc trưng chiều cao
    P10, P25, P50, P75, P95, P99 = np.percentile(points[:, 2], [10, 25, 50, 75, 95, 99])
    canopy_height_ratio = (P75 - P50) / (P99 - P25)

    # 📌 PCA Eigenvalues (Đánh giá hướng chính)
    cov_matrix = np.cov(points.T)
    eigenvalues = np.linalg.eigvals(cov_matrix)  # Tính eigenvalues
    eigenvalue_sum = np.sum(eigenvalues)  # Tổng tất cả eigenvalues
    eigenvalue_ratio = np.max(eigenvalues) / (eigenvalue_sum + 1e-10)  # Hướng chính của tán

    # 📌 Tính các đặc trưng dựa trên PCA
    if eigenvalue_sum > 0:
        linearity = eigenvalues[0] / eigenvalue_sum
        scatter = eigenvalues[2] / eigenvalue_sum
        eigentropy = -np.sum((eigenvalues / eigenvalue_sum) * np.log(eigenvalues / eigenvalue_sum + 1e-10))
    else:
        linearity = scatter = eigentropy = 0
    
    omnivariance = (eigenvalues[0] * eigenvalues[1] * eigenvalues[2]) ** (1/3) if np.all(eigenvalues > 0) else 0
    sum_of_eigenvalues = eigenvalue_sum
    planarity = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0] if eigenvalues[0] > 0 else 0
    sphericity = eigenvalues[2] / eigenvalues[0] if eigenvalues[0] > 0 else 0
    symmetry = eigenvalues[1] / eigenvalues[0] if eigenvalues[0] > 0 else 0

    # 📌 Đặc trưng cấu trúc
    mean_height = np.mean(points[:, 2]) - z_min
    std_height = np.std(points[:, 2])
    canopy_ratio = np.sum(points[:, 2] > np.median(points[:, 2])) / len(points)
    roughness = np.std(points[:, 2] - mean_z)
    normalized_height_ratio = mean_height / (z_max - z_min)

    # 📌 Convex Hull (Thể tích & Diện tích)
    try:
        hull = ConvexHull(points)
        convex_hull_volume = hull.volume
        convex_hull_area = hull.area
        
        # 📌 Tính bán kính Bounding Sphere (bán kính lớn nhất từ centroid đến điểm xa nhất)
        centroid = np.mean(points, axis=0)
        distances = np.linalg.norm(points - centroid, axis=1)
        bounding_sphere_radius = np.max(distances)
        
        # 📌 Tính Hull Compactness
        sphere_volume = (4/3) * np.pi * (bounding_sphere_radius ** 3)
        hull_compactness = convex_hull_volume / sphere_volume if sphere_volume > 0 else 0
    
    except:
        convex_hull_volume = 0  # Trường hợp không tính được
        convex_hull_area = 0
        hull_compactness = 0

    # 📌 Vertical Distribution (Phân bố điểm theo Z)
    hist_z, _ = np.histogram(points[:, 2], bins=10)
    vertical_distribution = np.std(hist_z) / np.mean(hist_z) if np.mean(hist_z) > 0 else 0

    # 📌 Point Entropy (Độ hỗn loạn của phân bố điểm)
    probs = hist_z / np.sum(hist_z) if np.sum(hist_z) > 0 else np.ones(10) / 10
    point_entropy = -np.sum(probs * np.log(probs + 1e-10))

    # 📌 Sự phân bố không gian của các đỉnh cao nhất
    top_points = points[points[:, 2] > np.percentile(points[:, 2], 90)]
    mean_x_top, mean_y_top, mean_z_top = np.mean(top_points, axis=0)
    var_x_top, var_y_top, var_z_top = np.var(top_points, axis=0)

    # 📌 Tính z_entropy
    z_probs = np.histogram(points[:, 2], bins=10, density=True)[0]
    z_entropy_value = -np.sum(z_probs * np.log(z_probs + 1e-10))
    pct_z_above_mean = np.sum(points[:, 2] > mean_z) / len(points) * 100
    pct_z_above_2 = np.sum(points[:, 2] > 2) / len(points) * 100

    # 📌 Độ rộng trung bình của tán cây
    tree_top = (mean_x, mean_y, z_max)
    crown_keypoints = extract_tree_crown_keypoints(points, tree_top, dis=0.5)
    num_crown_keypoints = len(crown_keypoints)
    avg_crown_width = np.mean(np.linalg.norm(crown_keypoints[:, :2] - np.array(tree_top[:2]), axis=1)) * 2 if num_crown_keypoints > 0 else 0

    # 📌 Tính diện tích biên của tán cây
    alpha_shape = compute_alpha_shape(points, alpha=0.1)
    crown_boundary_area = sum(poly.area for poly in alpha_shape.geoms) if isinstance(alpha_shape, MultiPolygon) else (alpha_shape.area if isinstance(alpha_shape, Polygon) else 0)
    
    # Height distribution features (Woods et al., 2008)
    out.update(
        {
            "width": width, "depth": depth, "height": height,
            "min_z": z_min, "max_z": z_max, "mean_z": mean_z, "var_z": var_z,
            "vertical_proportion": vertical_proportion,
            "density_xy": density_xy, "density_yz": density_yz, "density_xyz": density_xyz,
            "P10": P10, "P25": P25, "P50": P50, "P75": P75, "P95": P95, "P99": P99, "canopy_height_ratio": canopy_height_ratio,
            "eigenvalue_1": eigenvalues[0], "eigenvalue_2": eigenvalues[1], "eigenvalue_3": eigenvalues[2], "eigenvalue_sum": eigenvalue_sum, "eigenvalue_ratio": eigenvalue_ratio,
            "linearity": linearity, "scatter": scatter, "omnivariance": omnivariance, "eigentropy": eigentropy, "sum_of_eigenvalues": sum_of_eigenvalues, "planarity": planarity, "sphericity": sphericity, "symmetry": symmetry,
            "mean_height": mean_height, "std_height": std_height, "roughness": roughness,
            "convex_hull_volume": convex_hull_volume, "convex_hull_area": convex_hull_area,
            "vertical_distribution": vertical_distribution, "point_entropy": point_entropy,
            "top_points": top_points, "mean_x_top": mean_x_top, "mean_y_top": mean_y_top, "mean_z_top": mean_z_top,
            "var_x_top": var_x_top, "var_y_top": var_y_top, "var_z_top": var_z_top,
            "z_entropy_value": z_entropy_value, "pct_z_above_mean": pct_z_above_mean, "pct_z_above_2": pct_z_above_2,
            "avg_crown_width": avg_crown_width, "crown_boundary_area": crown_boundary_area, "alpha_shape": alpha_shape
        }
    )
    
    return out

In [ ]:
features = pd.json_normalize(data["path"].map(compute_cloud_features))
full = pd.concat([data, features], axis=1)

In [ ]:
full["mean_x_top_points"] = full["top_points"].apply(lambda pts: np.mean([p[0] for p in pts]))
full["mean_y_top_points"] = full["top_points"].apply(lambda pts: np.mean([p[1] for p in pts]))
full["mean_z_top_points"] = full["top_points"].apply(lambda pts: np.mean([p[2] for p in pts]))

full["var_x_top_points"] = full["top_points"].apply(lambda pts: np.var([p[0] for p in pts]))
full["var_y_top_points"] = full["top_points"].apply(lambda pts: np.var([p[1] for p in pts]))
full["var_z_top_points"] = full["top_points"].apply(lambda pts: np.var([p[2] for p in pts]))

from scipy.spatial.distance import pdist
full["max_top_distance"] = full["top_points"].apply(lambda pts: np.max(pdist(pts[:, :2])) if len(pts) > 1 else 0)

full["crown_boundary_area"] = full["alpha_shape"].apply(lambda poly: poly.area)
full["crown_perimeter"] = full["alpha_shape"].apply(lambda poly: poly.length)
full["crown_compactness"] = full["crown_boundary_area"] / full["crown_perimeter"]

In [ ]:
full.head()

In [ ]:
# Lấy ngẫu nhiên 1 mẫu từ mỗi species
sampled_data = full.groupby("species").sample(n=1, random_state=42)

# Hiển thị kết quả
sampled_data

In [ ]:
full.info()

In [ ]:
species_mapping = {
    "Fir": 1,      # Cây lãnh sam
    "Pine": 2,     # Cây thông
    "Spruce": 3,   # Cây vân sam
    "Alder": 4,    # Cây trăn
    "Aspen": 5,    # Cây dương
    "Birch": 6,    # Cây bạch dương
    "Tilia": 7     # Cây đoạn
}

full["species"] = full["species"].map(species_mapping)

In [ ]:
unknown_species = full[full["species"].isna()]["species"].unique()
print("Loài chưa ánh xạ:", unknown_species)

In [ ]:
X = full.drop(columns=["path", "species"]).copy()
y = full.pop("species")

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=68, shuffle=True)

In [ ]:
X_test.head()

# Multiclass clssification

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_selector
import numpy as np

In [ ]:
# Transformer trích xuất diện tích từ alpha_shape
class AlphaShapeFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([
            shape.area if shape is not None else 0  # Xử lý giá trị None
            for shape in X["alpha_shape"]
        ]).reshape(-1, 1)

# Transformer trích xuất trung bình độ cao từ top_points
class TopPointsFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([p[2] for p in points]) if isinstance(points, list) and len(points) > 0 else 0
            for points in X["top_points"]
        ]).reshape(-1, 1)

# ColumnTransformer xử lý tất cả cột số + trích xuất đặc trưng từ object
feature_extractor = ColumnTransformer([
    ("scale_numeric", StandardScaler(), make_column_selector(dtype_include=np.number)),  # Chuẩn hóa tất cả cột số
    ("extract_alpha_shape", AlphaShapeFeatureExtractor(), ["alpha_shape"]),  # Trích xuất từ alpha_shape
    ("extract_top_points", TopPointsFeatureExtractor(), ["top_points"])  # Trích xuất từ top_points
])

# Pipeline với Logistic Regression
pipe_lr = Pipeline([
    ("feature_extraction", feature_extractor),
    ("classifier", LogisticRegression(max_iter=10000, 
                                      multi_class="multinomial", 
                                      class_weight="balanced"))
])

In [ ]:
# Transformer trích xuất diện tích từ alpha_shape
class AlphaShapeFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([
            shape.area if shape is not None else 0  # Xử lý giá trị None
            for shape in X["alpha_shape"]
        ]).reshape(-1, 1)

# Transformer trích xuất trung bình độ cao từ top_points
class TopPointsFeatureExtractor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([p[2] for p in points]) if isinstance(points, list) and len(points) > 0 else 0
            for points in X["top_points"]
        ]).reshape(-1, 1)

# ColumnTransformer xử lý tất cả cột số + trích xuất đặc trưng từ object
feature_extractor = ColumnTransformer([
    ("scale_numeric", StandardScaler(), make_column_selector(dtype_include=np.number)),  # Chuẩn hóa tất cả cột số
    ("extract_alpha_shape", AlphaShapeFeatureExtractor(), ["alpha_shape"]),  # Trích xuất từ alpha_shape
    ("extract_top_points", TopPointsFeatureExtractor(), ["top_points"])  # Trích xuất từ top_points
])

# Pipeline với Logistic Regression
pipe_rf = Pipeline([
    ("feature_extraction", feature_extractor),
    ("classifier", RandomForestClassifier(n_estimators=1000, 
                           class_weight="balanced", 
                           random_state=42))
])

In [ ]:
%%time
# Cái này là giá của luồng pipe_lr
cv_result_multi = cross_validate(
    estimator=pipe_lr,
    X=X,
    y=y,
    cv=10,
    scoring="accuracy",
)

scores = cv_result_multi["test_score"]

fig, ax = plt.subplots()
ax.plot(scores, ls="", marker=".", c="k")
ax.set_ylim(0, 1.05)
ax.set_xlabel("Cross-validation iteration")
ax.set_ylabel("Accuracy")
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.axhline(scores.mean(), linestyle="dashed", alpha=0.3, c="k")
ax.annotate(
    text=f"Accuracy: {scores.mean():.3f} ± {scores.std():.3f}",
    xy=(4.5, 0.2),
    horizontalalignment="center",
    verticalalignment="center",
    bbox={
        "facecolor": "white",
    },
)
ax.set_title("Cross-validation results (multiclass)")
ax.grid(color="k", alpha=0.1)

In [ ]:
%%time
# Cái này là giá của luồng pipe_rf
cv_result_multi = cross_validate(
    estimator=pipe_rf,
    X=X,
    y=y,
    cv=10,
    scoring="accuracy",
)

scores = cv_result_multi["test_score"]

fig, ax = plt.subplots()
ax.plot(scores, ls="", marker=".", c="k")
ax.set_ylim(0, 1.05)
ax.set_xlabel("Cross-validation iteration")
ax.set_ylabel("Accuracy")
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
ax.axhline(scores.mean(), linestyle="dashed", alpha=0.3, c="k")
ax.annotate(
    text=f"Accuracy: {scores.mean():.3f} ± {scores.std():.3f}",
    xy=(4.5, 0.2),
    horizontalalignment="center",
    verticalalignment="center",
    bbox={
        "facecolor": "white",
    },
)
ax.set_title("Cross-validation results (multiclass)")
ax.grid(color="k", alpha=0.1)

In [ ]:
# Huấn luyện pipeline
pipe_lr.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)

In [ ]:
pred_multi_lr = pipe_lr.predict(X_test)
pred_multi_rf = pipe_rf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

# Tính Accuracy
accuracy = accuracy_score(y_test, pred_multi_lr)
print(f"Accuracy: {accuracy:.4f}")

# In báo cáo chi tiết Precision, Recall, F1-score
print(classification_report(y_test, pred_multi_lr, digits=4))

# Vẽ ma trận nhầm lẫn (Confusion Matrix)
cm = ConfusionMatrixDisplay.from_predictions(
    y_true=y_test,
    y_pred=pred_multi_lr,
    normalize="true",  # Bình thường hóa theo hàng (giá trị từ 0 -> 1)
)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

# Tính Accuracy
accuracy = accuracy_score(y_test, pred_multi_rf)
print(f"Accuracy: {accuracy:.4f}")

# In báo cáo chi tiết Precision, Recall, F1-score
print(classification_report(y_test, pred_multi_rf, digits=4))

# Vẽ ma trận nhầm lẫn (Confusion Matrix)
cm = ConfusionMatrixDisplay.from_predictions(
    y_true=y_test,
    y_pred=pred_multi_rf,
    normalize="true",  # Bình thường hóa theo hàng (giá trị từ 0 -> 1)
)

In [ ]:
# Kết hợp 2 mô hình bằng VotingClassifier
voting_clf = VotingClassifier(estimators=[
    ('lr', pipe_lr),
    ('rf', pipe_rf)
], voting='soft')  # 'soft' dùng xác suất dự đoán, 'hard' chỉ lấy kết quả đa số

# Train mô hình
voting_clf.fit(X_train, y_train)

# Testing

test_file_paths = glob.glob("/kaggle/input/hutechaichallenge2024-mc/Test/*/*.las")
test_file_paths.sort()

In [ ]:
test_data = pd.DataFrame({"path": test_file_paths})
test_data["species"] = test_data["path"].map(lambda p: p.split("/")[-2])
test_data.sample(n=3)

In [ ]:
test_data.info()

In [ ]:
features_test = pd.json_normalize(test_data["path"].map(compute_cloud_features))
full_test = pd.concat([test_data, features_test], axis=1)

In [ ]:
full_test["mean_x_top_points"] = full_test["top_points"].apply(lambda pts: np.mean([p[0] for p in pts]))
full_test["mean_y_top_points"] = full_test["top_points"].apply(lambda pts: np.mean([p[1] for p in pts]))
full_test["mean_z_top_points"] = full_test["top_points"].apply(lambda pts: np.mean([p[2] for p in pts]))

full_test["var_x_top_points"] = full_test["top_points"].apply(lambda pts: np.var([p[0] for p in pts]))
full_test["var_y_top_points"] = full_test["top_points"].apply(lambda pts: np.var([p[1] for p in pts]))
full_test["var_z_top_points"] = full_test["top_points"].apply(lambda pts: np.var([p[2] for p in pts]))

from scipy.spatial.distance import pdist
full_test["max_top_distance"] = full_test["top_points"].apply(lambda pts: np.max(pdist(pts[:, :2])) if len(pts) > 1 else 0)

full_test["crown_boundary_area"] = full_test["alpha_shape"].apply(lambda poly: poly.area)
full_test["crown_perimeter"] = full_test["alpha_shape"].apply(lambda poly: poly.length)
full_test["crown_compactness"] = full_test["crown_boundary_area"] / full_test["crown_perimeter"]

In [ ]:
full_test.head()

In [ ]:
# Lấy ngẫu nhiên 1 mẫu từ mỗi species
sampled_data_test = full_test.groupby("species").sample(n=1, random_state=42)

# Hiển thị kết quả
sampled_data_test

In [ ]:
full_test.info()

In [ ]:
full_test["species"] = full_test["species"].map(species_mapping)

In [ ]:
unknown_species_test = full_test[full_test["species"].isna()]["species"].unique()
print("Loài chưa ánh xạ:", unknown_species_test)

In [ ]:
X_test = full_test.drop(columns=["path", "species"]).copy()

In [ ]:
X_test.head()

In [ ]:
score_multi_test = voting_clf.predict_proba(X_test)
pred_multi_test = voting_clf.predict(X_test)

In [ ]:
y_test_true = full_test["species"].values

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, ConfusionMatrixDisplay

# Tính Accuracy
accuracy = accuracy_score(y_test_true, pred_multi_test)
print(f"Accuracy: {accuracy:.4f}")

# In báo cáo chi tiết Precision, Recall, F1-score
print(classification_report(y_test_true, pred_multi_test, digits=4))

# Vẽ ma trận nhầm lẫn (Confusion Matrix)
cm = ConfusionMatrixDisplay.from_predictions(
    y_true=y_test_true,
    y_pred=pred_multi_test,
    normalize="true",  # Bình thường hóa theo hàng (giá trị từ 0 -> 1)
)

In [ ]:
# Lấy tên file từ cột "path", bỏ phần mở rộng .las
file_names = full_test["path"].apply(lambda x: os.path.splitext(os.path.basename(x))[0])

# Tạo DataFrame submission
submission = pd.DataFrame({
    "name": file_names,
    "label": pred_multi_test
})

# Lưu file CSV
submission.to_csv("submission.csv", index=False)

print("File submission.csv đã được tạo thành công!")

In [ ]:
df_sub = pd.read_csv("/kaggle/working/submission.csv")
df_sub.info()